In [1]:
import h5py
import ipdb
import json
import os
import glob
import numpy as np
import pandas as pd
import scipy.io
from collections import defaultdict
import ipdb
from itertools import chain

'''
    UTILITIES
'''

forbidden_verbs = ['said']

def convertTime(value):
    second = value%60
    minute = int(value/60)%60
    hour = int(value/3600)
    return '{}:{:02}:{:02}'.format(hour, minute, second)

def getBookName(filen):
    filen.split('/')[-1]
    book_path = '{}/books/'.format(bksnmvs_path)
    book_name = filen+'.(hl-2).mat'
    return book_path+book_name



In [2]:
'''
    PATHS AND NAMES
'''

dataset_path = '/data/vision/torralba/datasets/movies/data/'
bksnmvs_path = '/data/vision/torralba/frames/data_acquisition/booksmovies/data/booksandmovies/'
anno_path = '{}/antonio/annotation/'.format(bksnmvs_path)
text_annotation_path = '/data/vision/torralba/movies-books/booksandmovies/joanna/bksnmovies/data/gt_alignment/consecutive_text_labels_v2'
labeled_sents = json.load(open('../data/bksnmovies/labeled_sents.json', 'r'))
movies = ['American.Psycho','Brokeback.Mountain','Fight.Club','Gone.Girl','Harry.Potter.and.the.Sorcerers.Stone','No.Country.for.Old.Men','One.Flew.Over.the.Cuckoo.Nest','Shawshank.Redemption','The.Firm','The.Green.Mile','The.Road']
movies_titles = [movie.replace('.', '_') for movie in movies]
imbds = ['tt0144084','tt0388795','tt0137523','tt2267998','tt0241527','tt0477348','tt0073486','tt0111161','tt0106918','tt0120689','tt0898367']
dataset_split = 1 # 1 for 90% data from all movies in train and 10% in val; 2 for n-1 movies in train 1 in val
val_movie = movies[0]

labeled_sents = json.load(open('../data/bksnmovies/labeled_sents.json', 'r'))


In [3]:
i = 3
movie =movies[i]
print(movie)

title = movies_titles[i]
imbd = imbds[i]

labels = labeled_sents[movie]
### SCENE AND SHOTS

# Load shot info
srt = scipy.io.loadmat('{}srts/{}.mat'.format(bksnmvs_path, movie))


### GT DATAFRAME

# Load book
book_name = getBookName(movie)
book_file = scipy.io.loadmat(book_name)
sentences = book_file['book']['sentences'].item()['sentence']
for i in range(420, 500):
    print(f"{i}: {sentences[i][0][0]}")

Gone.Girl
420: This place is a bit of a hole in the wall,.
421: But we had a great kiss there one Tuesday last fall.
422: Ever been in a spelling bee as a kid?
423: That snowy second after the announcement of the word as you sift your brain to see if you can spell it?
424: It was like that, the blank panic.
425: "An Irish bar in a not-so-Irish place," Amy nudged.
426: I bit the side of my lip, started a shrug, scanning our living room as if the answer might appear.
427: She gave me another very long minute.
428: "We were lost in the rain," she said in a voice that was pleading on the way to peeved.
429: I finished the shrug.
430: "McMann's, Nick.
431: Remember, when we got lost in the rain in Chinatown trying to find that dim sum place, and it was supposed to be near the statue of Confucius but it turns out there are two statues of Confucius, and we ended up at that random Irish bar all soaking wet, and we slammed a few whiskeys, and you grabbed me and kissed me, and it was""Right!
43

In [4]:
repr(sentences[420][0][0])

"'This place is a bit of a hole in the wall,.'"

In [5]:
def isVisualDialog(labels):
    is_dialog = False
    is_visual = False
    for label in labels:
        if 'dialog' in label: is_dialog = True
        if 'description' in label: is_visual = True
    if is_visual and is_dialog: return ['dialog', 'visual']
    else: return ['dialog']

def isVisual(labels):
    is_visual = False
    for label in labels:
        if 'description' in label: is_visual = True
    if is_visual: return ['visual']
    else: return ['non-visual']


In [6]:
# Ground-truth annotations
dialog_anno = {}
i = 7
movie = movies[i]
print('Getting ground-truth for movie: ' + movie)

title = movies_titles[i]
imbd = imbds[i]

labels = labeled_sents[movie]
### SCENE AND SHOTS

# Load shot info
srt = scipy.io.loadmat('{}srts/{}.mat'.format(bksnmvs_path, movie))


### GT DATAFRAME

# Load book
book_name = getBookName(movie)
book_file = scipy.io.loadmat(book_name)

dialog_anno[movie] = {}

# Filter out dialogs
sentences = book_file['book']['sentences'].item()['sentence']
is_dialog = False
for j, sentence in enumerate(sentences):
    count = sentence[0][0].count('"')
    labels = labeled_sents[movie][str(j)]
    if count%2 == 1: # if there is an odd number of "
        if is_dialog: is_dialog = False # if there was a dialog going on, it has been closed
        else: is_dialog = True # otherwise, a dialog has been opened
        dialog_anno[movie][j] = isVisualDialog(labels)
    elif count == 0: # if there is no "
        if is_dialog: dialog_anno[movie][j] = ['dialog'] # if there was a dialog going on, still is
        else: dialog_anno[movie][j] = isVisual(labels) # otherwise, no dialog
    else: # if there is an even number of "
        is_dialog = False # dialogs have been opened and closed
        dialog_anno[movie][j] = isVisualDialog(labels)

Getting ground-truth for movie: Shawshank.Redemption


In [7]:
for j in range(100,200):
    print(f"{sentences[j][0][0]}: {dialog_anno[movie][j]}")

A clerk from the Wise Pawnshop in Lewiston testified that he had sold a six-shot.38 Police Special to Andrew Dufresne just two days before the double murder.: ['non-visual']
A bartender from the country club bar testified that Andy had come in around seven o'clock on the evening of September 10th, had tossed off three straight whiskeys in a twenty-minute period-when he got up from the bar-stool he told the bartender that he was going up to Glenn Quentin's house and he, the bartender, could "read about the rest of it in the papers.": ['dialog']
Another clerk, this one from the Handy-Pik store a mile or so from Quentin's house, told the court that Dufresne had come in around quarter to nine on that same night.: ['non-visual']
He purchased cigarettes, three quarts of beer, and some dishtowels.: ['visual']
The county medical examiner testified that Quentin and the Dufresne woman had been killed between 11:00 P.M. and 2:00 A.M. on the night of September 10th- 11th.: ['non-visual']
The detec